## 1. Make descriptions -> generate images

In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

import gc
import textwrap
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
os.rename('/content/drive/MyDrive/хонор проект/БПМ-24-4.zip', '/content/drive/MyDrive/хонор проект/data.zip')
!unzip '/content/drive/MyDrive/хонор проект/data.zip'
os.rename('/content/БПМ-24-4', '/content/data')

In [ ]:
path = '/content/data/stickers'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device

In [ ]:
import textwrap

In [ ]:
df = pd.read_csv('/content/data/output.csv', encoding='utf-16')
df['Last Words'] = df['Last Words'].map(lambda x: ' '.join(x[-512:].split()[1:]))# textwrap.wrap(x, width=512))

In [ ]:
height, width = 512, 512

### captioning

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True)

In [ ]:
import gc
gc.collect()
del model
del processor

In [ ]:
task_prompt = "<MORE_DETAILED_CAPTION>" # "<DETAILED_CAPTION>", "<CAPTION>"
# text_input =
prompt = task_prompt# + text_input

In [ ]:
def caption_sticker(sample):
  image_path = os.path.join(path, sample)
  raw_image = Image.open(os.path.join(path, image_path)).convert('RGB')
  inputs = processor(text=prompt, images=raw_image, return_tensors="pt").to(device, torch_dtype)
  generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    num_beams=3
  )
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
  parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(raw_image.width, raw_image.height))
  return parsed_answer[prompt]

In [ ]:
df['caption'] = df['Sticker Name'].progress_apply(caption_sticker)

In [ ]:
from datasets import Dataset
ds = Dataset.from_pandas(df[['Last Words', 'caption']])
ds.push_to_hub('stickers_captioning', token='')

### image generation

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16)
pipe.enable_model_cpu_offload()

In [ ]:
height, width = raw_image.height, raw_image.width

In [ ]:
negative_prompt = "low quality, bad quality"
image = pipe(prompt=caption, negative_prompt=negative_prompt, prior_guidance_scale =1.0, height=height, width=width).images[0]
image

In [ ]:
caption